# Module 1 Load Data

In [1]:
import sys
from collections import defaultdict
import numpy as np
import pandas as pd
import tensorflow as tf

## load table 

In [2]:
## load dataset & filter out missing data
# dbfile = pd.read_csv('./Data/vdjdb.slim.txt', sep='\t')
dbfile = pd.read_csv('./Data/VDJDB/vdjdb_full.txt', sep='\t')
# print(dbfile.columns)
dbfile = dbfile[['cdr3.alpha', 'cdr3.beta','antigen.epitope','antigen.species',"mhc.class"]]
print(dbfile.shape)
dbfile = dbfile.dropna(subset=['cdr3.alpha', 'cdr3.beta','antigen.epitope'])
print(dbfile.shape)
dbfile.head()

(60055, 5)
(29266, 5)


C:\Users\croco\AppData\Local\Temp\ipykernel_5352\3075150544.py:3: DtypeWarning: Columns (20,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dbfile = pd.read_csv('./Data/VDJDB/vdjdb_full.txt', sep='\t')


,cdr3.alpha,cdr3.beta,antigen.epitope,antigen.species,mhc.class
0,CIVRAPGRADMRF,CASSYLPGQGDHYSNQPQHF,FLKEKGGL,HIV-1,MHCI
2,CAVPSGAGSYQLTF,CASSFEPGQGFYSNQPQHF,FLKEKGGL,HIV-1,MHCI
3,CAVKASGSRLT,CASSYEPGQVSHYSNQPQHF,FLKEKGGL,HIV-1,MHCI
4,CAYRPPGTYKYIF,CASSALASLNEQFF,FLKEKGGL,HIV-1,MHCI
5,CIVRAPGRADMRF,CASSYLPGQGDHYSNQPQHF,FLKEQGGL,HIV-1,MHCI


In [3]:
iedbfile = pd.read_csv('./Data/IEDB/tcell_receptor_table_export_1668026760.csv')
# print(iedbfile.columns)
# Chain 1 Full Sequence', 'Chain 1 Accession', 'Chain 1 CDR3 Curated',
#        'Chain 1 CDR3 Calculated'
iedbfile = iedbfile[['Chain 1 CDR3 Curated','Chain 2 CDR3 Curated',"Description","Organism"]]
print(iedbfile.shape)
iedbfile = iedbfile.dropna(subset=['Chain 1 CDR3 Curated','Chain 2 CDR3 Curated',"Description"])
iedbfile.Description = iedbfile.Description.str.split(' ').str[0]
iedbfile = iedbfile[~iedbfile.Description.str.contains('[^ARNDCQEGHILKMFPSTWYV]',na=False)]
print(iedbfile.shape)
iedbfile.head()

C:\Users\croco\AppData\Local\Temp\ipykernel_5352\605436328.py:1: DtypeWarning: Columns (8,10,11,13,15,16,17,18,19,20,21,22,23,24,25,30,31,36,37,42,44,45,46,47,48,49,50,51,52,53,54,59,60,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  iedbfile = pd.read_csv('./Data/IEDB/tcell_receptor_table_export_1668026760.csv')


(205824, 4)
(25927, 4)


,Chain 1 CDR3 Curated,Chain 2 CDR3 Curated,Description,Organism
0,IVVRSSNTGKLI,ASSQDRDTQY,VMAPRTLIL,Homo sapiens (human)
80,CAVTTDSWGKLQF,CASRPGLAGGRPEQYF,LLFGYPVYV,Human T-cell leukemia virus type I (Human T ce...
82,CAVTTDSWGKLQF,CASRPGLMSAQPEQYF,LLFGYPVYV,Human T-cell leukemia virus type I (Human T ce...
109,CAVRPTSGGSYIPTF,CASSYVGNTGELFF,SLLMWITQC,Homo sapiens (human)
202,TVYGGATNKLI,SARGGSYNSPLH,LSRFSWGAEGQRPGFGYGG,Homo sapiens (human)


In [4]:
# mixeddb = 

## Encoding & split sets

In [5]:
from functions import load
# from functions.load import enc_list_bl_max_len, positional_coding, blosum50_20aa 

In [265]:
# from importlib import reload
reload(load)

<module 'functions.load' from 'c:\\Users\\croco\\OneDrive - Emory University\\Courses\\BMI 536 Deep Learning\\FinalProj\\functions\\load.py'>

In [7]:
print(dbfile.head(n=2)["cdr3.alpha"])
load.enc_list_bl_max_len(dbfile.head(n=2)["cdr3.alpha"],load.blosum50_20aa,30).shape

0     CIVRAPGRADMRF
2    CAVPSGAGSYQLTF
Name: cdr3.alpha, dtype: object


(2, 30, 20)

In [249]:
tokens = load.blosum50_20aa.keys()

tokens = list(tokens)
tokens.insert(0, '.')
tokens.insert(1, '>')
tokens.insert(2, '<')
tk_dict = defaultdict(lambda: None)
for idx in range(len(tokens)):
    tk_dict[tokens[idx]]=idx
vocabSize=23

In [250]:
load.positional_coding(dbfile.head(n=52)["cdr3.alpha"],tk_dict,30).shape
load.positional_coding(iedbfile.head(n=52)["Chain 1 CDR3 Curated"],tk_dict,30).shape

(52, 30)

In [252]:
from sklearn.model_selection import train_test_split

In [253]:
train, test = train_test_split(dbfile[dbfile['antigen.epitope'].str.len()<=12], shuffle=True ,test_size=0.2)

In [254]:
np.concatenate(load.positional_coding(("CIVRAPGRADMRF","CASSYLPGQGDHYSNQPQHF"), tk_dict, 30)) #CIVRAPGRADMRF	CASSYLPGQGDHYSNQPQHF	FLKEKGGL	

array([ 7., 12., 22.,  4.,  3., 17., 10.,  4.,  3.,  6., 15.,  4., 16.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  7.,  3., 18., 18., 21., 13., 17., 10.,  8.,
       10.,  6., 11., 21., 18.,  5.,  8., 17.,  8., 11., 16.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

## to datasets

In [266]:
ept_train = load.ept_coding(train['antigen.epitope'], tk_dict, 13,train=True)
tcra_train = load.positional_coding(train["cdr3.alpha"], tk_dict, 30)
tcrb_train = load.positional_coding(train["cdr3.beta"], tk_dict, 30)
# ept_train = np.roll(ept_train, 1)
ept_train_lb = load.ept_coding(train['antigen.epitope'], tk_dict, 13,label=True)
tcr_train = np.concatenate((tcra_train,tcrb_train),axis=1)
print(ept_train.shape, tcra_train.shape, tcrb_train.shape,tcr_train.shape)
ept_test = load.ept_coding(test['antigen.epitope'], tk_dict, 13,train=True)
tcra_test = load.positional_coding(test["cdr3.alpha"], tk_dict, 30)
tcrb_test = load.positional_coding(test["cdr3.beta"], tk_dict, 30)
tcr_test = np.concatenate((tcra_test,tcrb_test),axis=1)
# ept_train = np.roll(ept_train, 1)
ept_test_lb = load.ept_coding(test['antigen.epitope'], tk_dict, 13,label=True)

(23161, 13) (23161, 30) (23161, 30) (23161, 60)


In [267]:
tcr_train = tf.convert_to_tensor(tcr_train)
ept_train = tf.convert_to_tensor(ept_train)
ept_train_lb = tf.convert_to_tensor(ept_train_lb)
tcr_test = tf.convert_to_tensor(tcr_test)
ept_test = tf.convert_to_tensor(ept_test)
ept_test_lb = tf.convert_to_tensor(ept_test_lb)

# Model Input

In [226]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [227]:
from functions import model
# reload(model)

In [257]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
MAX_TOKENS = 32


def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .prefetch(buffer_size=tf.data.AUTOTUNE))

In [268]:
dataset = tf.data.Dataset.from_tensor_slices(((tcr_train,ept_train),ept_train_lb))
dataset = make_batches(dataset)
dataset_val = tf.data.Dataset.from_tensor_slices(((tcr_test, ept_test),ept_test_lb))
dataset_val = make_batches(dataset_val)

In [269]:
for (tcr, ept), ept_lb in dataset.take(2):
  break

print(tcr.shape)
print(ept.shape)
print(ept_lb.shape)

(64, 60)
(64, 13)
(64, 13)


In [270]:
print(ept[0])
print(ept_lb[0])

embed_tcr = model.PositionalEmbedding(vocab_size=vocabSize, d_model=512)
embed_ept = model.PositionalEmbedding(vocab_size=vocabSize, d_model=512)

tcr_emb = embed_tcr(tcr)
ept_emb = embed_ept(ept)
# print(tcr_emb._keras_mask)

tf.Tensor([ 1. 19. 18.  3. 15.  8. 19. 15. 13. 16.  0.  0.  0.], shape=(13,), dtype=float64)
tf.Tensor([19. 18.  3. 15.  8. 19. 15. 13. 16.  2.  0.  0.  0.], shape=(13,), dtype=float64)


In [271]:
sample_encoder = model.Encoder(num_layers=4,
                         d_model=512,
                         num_heads=8,
                         dff=2048,
                         vocab_size=vocabSize)

sample_encoder_output = sample_encoder(tcr, training=False)
print(tcr.shape)
print(sample_encoder_output.shape)

(64, 60)
(64, 60, 512)


In [272]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [273]:
transformer = model.Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=np.int64(vocabSize),
    target_vocab_size=np.int64(vocabSize),
    dropout_rate=dropout_rate)

In [274]:
output = transformer((tcr, ept))

print(tcr.shape)
print(ept.shape)
print(output.shape)

attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

(64, 60)
(64, 13)
(64, 13, 23)
(64, 8, 13, 60)


In [275]:
# output[1,1,:]
# ept

In [276]:
learning_rate = model.CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

# plt.plot(learning_rate(tf.range(40000, dtype=tf.float32)))
# plt.ylabel('Learning Rate')
# plt.xlabel('Train Step')

In [277]:
transformer.compile(
    loss=model.masked_loss,
    optimizer=optimizer,
    metrics=[model.masked_accuracy])

In [278]:
transformer.summary()

Model: "transformer_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_10 (Encoder)        multiple                  2641792   
                                                                 
 decoder_4 (Decoder)         multiple                  4752768   
                                                                 
 dense_132 (Dense)           multiple                  2967      
                                                                 
Total params: 7,397,527
Trainable params: 7,397,527
Non-trainable params: 0
_________________________________________________________________


In [279]:
transformer.fit(dataset,
                epochs=20,
                validation_data=dataset_val)

Epoch 1/20
362/362 [==============================] - 33s 73ms/step - loss: 1.4906 - masked_accuracy: 0.6144 - val_loss: 0.7131 - val_masked_accuracy: 0.8282
Epoch 2/20
362/362 [==============================] - 25s 69ms/step - loss: 0.5786 - masked_accuracy: 0.8572 - val_loss: 0.4575 - val_masked_accuracy: 0.8841
Epoch 3/20
362/362 [==============================] - 25s 69ms/step - loss: 0.4241 - masked_accuracy: 0.8923 - val_loss: 0.3852 - val_masked_accuracy: 0.9007
Epoch 4/20
362/362 [==============================] - 25s 70ms/step - loss: 0.3752 - masked_accuracy: 0.9038 - val_loss: 0.3597 - val_masked_accuracy: 0.9075
Epoch 5/20
362/362 [==============================] - 25s 70ms/step - loss: 0.3526 - masked_accuracy: 0.9091 - val_loss: 0.3421 - val_masked_accuracy: 0.9145
Epoch 6/20
362/362 [==============================] - 25s 70ms/step - loss: 0.3403 - masked_accuracy: 0.9124 - val_loss: 0.3378 - val_masked_accuracy: 0.9132
Epoch 7/20
362/362 [==============================] 

## Translator

In [280]:
reload(model)

<module 'functions.model' from 'c:\\Users\\croco\\OneDrive - Emory University\\Courses\\BMI 536 Deep Learning\\FinalProj\\functions\\model.py'>

In [281]:
translator = model.Translator(tk_dict, tokens, transformer)

In [282]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [283]:
sentence = ("CIVRAPGRADMRF","CASSYLPGQGDHYSNQPQHF")		
ground_truth = "FLKEKGGL"

translated_text, attention_weights = translator(sentence)
print_translation(sentence, translated_text, ground_truth)

Input:         : ('CIVRAPGRADMRF', 'CASSYLPGQGDHYSNQPQHF')
Prediction     : ['>KLGGALQAK<<<']
Ground truth   : FLKEKGGL


In [285]:
tf.saved_model.save(transformer, "models/mdl_vdj_1")
# model = tf.saved_model.load("models/mdl_vdj_1")

INFO:tensorflow:Assets written to: models/mdl_vdj_1\assets


INFO:tensorflow:Assets written to: models/mdl_vdj_1\assets
